In [4]:
import re
import os
import glob
import logging
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Constants for write-back detection (permanent libs)
PERMANENT_LIBS = [r"^prod\.", r"^finance\.", r"^analytics\."]


def read_sas_files(folder_path):
    """
    Recursively read all .sas files in the given folder.
    Returns a dict: {file_path: file_content}
    """
    sas_files = glob.glob(os.path.join(folder_path, '**', '*.sas'), recursive=True)
    files = {}
    for fp in sas_files:
        try:
            with open(fp, 'r', encoding='utf-8', errors='ignore') as f:
                files[fp] = f.read()
        except Exception as e:
            logging.warning(f"Failed to read {fp}: {e}")
    return files


def strip_comments(code):
    """
    Remove SAS comments (*...; and /*...*/), preserving inline code.
    """
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL | re.IGNORECASE)
    code = re.sub(r'\*[^;]*;', '', code, flags=re.IGNORECASE)
    return code


def find_libnames(code):
    """
    Extract declared librefs from LIBNAME statements.
    Returns set of libref names.
    """
    pattern = re.compile(r"\bLIBNAME\s+(?P<lib>\w+)\s*=", re.IGNORECASE)
    return set(m.group('lib').lower() for m in pattern.finditer(code))


def find_includes(code, file_dir):
    """
    Extract %INCLUDE paths (single/double) and macro-based paths.
    Returns list of tuples (path, exists_boolean).
    """
    includes = []
    # match single or double quoted
    pattern = re.compile(r"%INCLUDE\s+(?:'|\")(?P<path>[^'\"]+)(?:'|\")", re.IGNORECASE)
    for m in pattern.finditer(code):
        path = m.group('path')
        full = os.path.join(file_dir, path)
        includes.append((path, os.path.exists(full)))
    return includes


def detect_blocks(code):
    """
    Identify and isolate PROC, DATA, MACRO blocks with specific end markers.
    Returns list of dicts with keys: type, name, raw_code.
    """
    blocks = []

    # 1. MACRO definitions
    for m in re.finditer(r"%MACRO\s+(?P<name>\w+)(?P<params>\([^)]*\))?;", code, flags=re.IGNORECASE):
        start = m.start()
        # find corresponding %MEND
        end_match = re.search(rf"%MEND\s+{re.escape(m.group('name'))};", code[m.end():], flags=re.IGNORECASE)
        end = m.end() + (end_match.end() if end_match else 0)
        raw = code[start:end]
        blocks.append({'type': 'MACRO', 'name': m.group('name'), 'raw': raw})

    # 2. DATA steps
    for m in re.finditer(r"\bDATA\s+(?P<name>[\w\.]+)\s*;", code, flags=re.IGNORECASE):
        start = m.start()
        # DATA steps end with RUN;
        end_match = re.search(r"\bRUN;", code[m.end():], flags=re.IGNORECASE)
        end = m.end() + (end_match.end() if end_match else 0)
        raw = code[start:end]
        blocks.append({'type': 'DATA', 'name': m.group('name'), 'raw': raw})

    # 3. PROC steps
    proc_pattern = re.compile(r"\bPROC\s+(?P<name>\w+)\b", flags=re.IGNORECASE)
    for m in proc_pattern.finditer(code):
        start = m.start()
        name = m.group('name').upper()
        rest = code[m.end():]
        # interpret end based on PROC
        if name == 'SQL':
            end_match = re.search(r"\bQUIT;", rest, flags=re.IGNORECASE)
        else:
            end_match = re.search(r"\bRUN;", rest, flags=re.IGNORECASE)
        end = m.end() + (end_match.end() if end_match else 0)
        raw = code[start:end]
        blocks.append({'type': 'PROC', 'name': name, 'raw': raw})

    return blocks


def extract_tables_for_block(block):
    """
    Extract input and output tables depending on block type with tailored patterns.
    Returns two sets: inputs, outputs.
    """
    raw = block['raw']
    inputs, outputs = set(), set()

    if block['type'] == 'DATA':
        # SET, MERGE, IN=, etc.
        for pat in [r"\bSET\s+(?P<table>[\w\.]+)",
                    r"\bMERGE\s+(?P<table>[\w\.]+)",
                    r"\bIN=\s*(?P<table>[\w\.]+)"]:
            for m in re.finditer(pat, raw, flags=re.IGNORECASE):
                inputs.add(m.group('table').lower())
        # OUTPUT dataset for DATA step is the name after DATA statement
        data_name = re.search(r"\bDATA\s+(?P<name>[\w\.]+)\s*;", raw, flags=re.IGNORECASE)
        if data_name:
            outputs.add(data_name.group('name').lower())

    elif block['type'] == 'PROC' and block['name'] == 'SQL':
        # PROC SQL: FROM, JOIN, INTO, CREATE TABLE
        for pat in [r"\bFROM\s+(?P<table>[\w\.]+)",
                    r"\bJOIN\s+(?P<table>[\w\.]+)",
                    r"\bINTO\s+(?P<table>[\w\.]+)",
                    r"\bCREATE\s+TABLE\s+(?P<table>[\w\.]+)"]:
            for m in re.finditer(pat, raw, flags=re.IGNORECASE):
                if pat.startswith("FROM") or pat.startswith("JOIN"):
                    inputs.add(m.group('table').lower())
                else:
                    outputs.add(m.group('table').lower())

    else:
        # Generic PROC: look for OUT=, DATA= for output, any TABLE= for input if needed
        for pat in [r"\bOUT=\s*(?P<table>[\w\.]+)",
                    r"\bDATA=\s*(?P<table>[\w\.]+)"]:
            for m in re.finditer(pat, raw, flags=re.IGNORECASE):
                outputs.add(m.group('table').lower())

    return inputs, outputs


def detect_write_back(tables):
    """
    Returns True if any table write references a permanent lib.
    """
    for tbl in tables:
        for lib in PERMANENT_LIBS:
            if re.match(lib, tbl, flags=re.IGNORECASE):
                return True
    return False


def comment_write_backs(raw_code):
    """
    Comment out lines referencing permanent libraries.
    """
    lines = raw_code.splitlines()
    out = []
    for ln in lines:
        if any(re.search(lib, ln, flags=re.IGNORECASE) for lib in PERMANENT_LIBS):
            out.append(f"* {ln}  /* commented out by migration tool */")
        else:
            out.append(ln)
    return "\n".join(out)


def parse_file(path, content):
    """
    Parse a single SAS file and return list of metadata dicts for each block.
    """
    clean = strip_comments(content)
    librefs = find_libnames(clean)
    incs = find_includes(clean, os.path.dirname(path))
    blocks = detect_blocks(clean)
    records = []

    for b in blocks:
        try:
            inp, outp = extract_tables_for_block(b)
            write_flag = 'Yes' if detect_write_back(outp | inp) else 'No'
            missing_libs = [tbl.split('.')[0] for tbl in (inp | outp)
                            if tbl.split('.')[0] not in librefs and tbl.split('.')[0] != 'work']
            missing_includes = 'Yes' if any(not ex for _, ex in incs) else 'No'
            raw_code = comment_write_backs(b['raw']) if write_flag == 'Yes' else b['raw']

            records.append({
                'file_name': os.path.basename(path),
                'block_type': b['type'],
                'block_name': b.get('name'),
                'input_tables': list(inp) or None,
                'output_tables': list(outp) or None,
                'write_back': write_flag,
                'missing_librefs': missing_libs or None,
                'missing_includes': missing_includes,
                'raw_code': raw_code
            })
        except Exception as e:
            logging.error(f"Error parsing block {b.get('name')} in {path}: {e}")
    return records


def main(folder):
    files = read_sas_files(folder)
    all_records = []
    for path, content in files.items():
        all_records.extend(parse_file(path, content))

    df = pd.DataFrame(all_records)
    out_folder = os.path.join(folder, 'excel')
    os.makedirs(out_folder, exist_ok=True)
    out_path = os.path.join(out_folder, 'sas_migration_report2.xlsx')
    df.to_excel(out_path, index=False)
    logging.info(f"Report written to {out_path}")


if __name__ == '__main__':


    folder = "../data" 
    main(folder)


2025-07-10 12:38:26,084 - INFO - Report written to ../data\excel\sas_migration_report2.xlsx
